# Creating and updating a library
*Jonathan Bac, 03/19*

In this notebook we use the package DeepMod and a simple PINN+library to recover the terms and coefficients of viscoelastic models, from two parameter models up to 3 element generalized Maxwell

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import seaborn as sns
sns.set_style('darkgrid')

def map_to_coeff_vector(mask, coeffs):
    coeffs_vector = np.copy(coeffs) 
    w = np.zeros_like(mask)
    for idx in np.arange(w.shape[0]):
          if mask[idx, 0]==1:
            w[idx, 0] = coeffs_vector[0]
            coeffs_vector = coeffs_vector[1:]
    return w

ModuleNotFoundError: No module named 'tensorflow'

## Creating data

Set Parameters

In [43]:
from scipy.io import loadmat

In [44]:
data = loadmat('Order_2_Tau_5_strain.mat')

In [45]:
usol = np.real(data['Expression1'])
usol= usol.reshape((5981,3))

In [33]:
X = np.expand_dims(usol[:,0],axis=1)
y_strain = np.expand_dims(usol[:,1],axis=1)
y_stress = np.expand_dims(usol[:,2],axis=1)

Examples running DeepMod and old model selection on a single trajectory

In [37]:
idx = np.random.choice(y_stress.size, y_stress.size)
split = 1000
X_train = X[idx[:split]]
y_train = y_stress[idx[:split]]

## Defining PINN

We again define a PINN, but this time we define two new variables:
* sparsity_mask - This is a $(N\times1)$ vector of 0's and 1's, indicating which of the terms are to be used in the fitting procedure. One means the term is active, zero means inactive. If we have sixteen terms in the initial library, $N=16$.
* parameters - This is the weights vector which is optimized. It's size is dependent on the number of active terms in the sparsity mask. In previous implementations, each coefficient had its own variable; with >5 variables this becomes unusable and thus we now have a vector. Using matrix multiplication on the library yields the same result.

For now we hard code a 16 term initial library, we'll show in a bit how this is determined.

In [47]:
#DeepMod
import sys
from deepymod.DeepMoD import DeepMoD
from deepymod.library_functions import mech_library_stress
from deepymod.utilities import * 

In [48]:
layers = [1, 40, 40, 40, 40, 40, 1]
l1 = 10**(-7)
seeds = [2, 5, 12, 34, 40]

In [49]:
y = np.expand_dims(usol[:,2],axis=1)
y_train = y[idx[:split]]
np.random.seed(42)

In [50]:
config = {'layers': layers, 'lambda': l1}
library_config = {'max_order': 3, 'total_terms': 7}
train_opts = {'max_iterations': 100000, 'grad_tol':10**-7, 'learning_rate': 0.002, 'beta1': 0.99, 'beta2': 0.999, 'epsilon': 10**-8}
output_opts = {'output_directory': 'stress_new/'}
sparse_vectors = DeepMoD(X_train, y_train, config, mech_library_stress, library_config, train_opts, output_opts)

Epoch | Total loss | Loss gradient | MSE | PI | L1 
0 [0.7932092, 0.022226999, 0.59966934, 0.19353873, 1.1404732e-06]
500 [0.037720338, 0.01638238, 0.027051069, 0.01066906, 2.0899127e-07]
1000 [0.017308433, 0.003560263, 0.012219654, 0.0050885538, 2.2429738e-07]
1500 [0.011332673, 0.0016492119, 0.0071367905, 0.0041956473, 2.3523906e-07]
2000 [0.0055442853, 0.0030344692, 0.0034507886, 0.0020932101, 2.8701092e-07]
2500 [0.00335836, 0.0016631685, 0.0022135852, 0.0011445207, 2.5397023e-07]
3000 [0.0029975637, 0.0009685497, 0.0020897312, 0.00090760604, 2.2654639e-07]
3500 [0.002701183, 0.00058609975, 0.001980575, 0.0007204089, 1.9918643e-07]
4000 [0.002448006, 0.00037934267, 0.0018326037, 0.00061522855, 1.7366071e-07]
4500 [0.0021743043, 0.00034562225, 0.0016415667, 0.0005325882, 1.4948239e-07]
5000 [0.0016831665, 0.00036405204, 0.0011891425, 0.00049387384, 1.5021803e-07]
5500 [0.0009795512, 0.00048236045, 0.00033288836, 0.00064649974, 1.6313767e-07]
6000 [0.00077906065, 0.000710376, 0.00026

KeyboardInterrupt: 

In [33]:
# %%Setting up and running
for noise in noise_levels:
    results= []
    for seed in seeds:
        np.random.seed(seed)
        tf.set_random_seed(seed)
        y = np.expand_dims(usol[:,1],axis=1)
        print(noise, seed)
        y_noisy = y + noise * np.std(y) * np.random.randn(y.size, 1)
        y_train = y_noisy[idx[:split]]
        config = {'layers': layers, 'lambda': l1}
        library_config = {'max_order': 3, 'total_terms': 7}
        train_opts = {'max_iterations': 100000, 'grad_tol':10**-7, 'learning_rate': 0.002, 'beta1': 0.99, 'beta2': 0.999, 'epsilon': 10**-8}
        output_opts = {'output_directory': 'double_a_noise_2500/'}
        sparse_vectors = DeepMoD(X_train, y_train, config, mech_library, library_config, train_opts, output_opts)
        results.append(sparse_vectors)
    np.save('double_a_noise_2500_'+str(noise)+'.npy',results)
# %%Printing output

0.05 2
Epoch | Total loss | Loss gradient | MSE | PI | L1 
0 [0.27925655, 0.001080351, 0.26362142, 0.01563273, 2.4028056e-06]
500 [0.015873794, 0.005225159, 0.013667662, 0.0022058769, 2.5561977e-07]
1000 [0.010611314, 0.0017351551, 0.009932939, 0.000678093, 2.8228894e-07]
1500 [0.009410972, 0.0007762281, 0.0089362655, 0.00047440757, 2.993764e-07]
2000 [0.008723166, 0.00029198988, 0.008342804, 0.000380059, 3.0270323e-07]
2500 [0.008428179, 0.0004751908, 0.008045916, 0.00038196475, 2.9705566e-07]
3000 [0.008264336, 0.0005237465, 0.007885626, 0.00037842034, 2.8989663e-07]
3500 [0.008037664, 0.0005410761, 0.0076278877, 0.00040949188, 2.8431714e-07]
4000 [0.007835829, 0.0004978138, 0.007486646, 0.0003489043, 2.7839212e-07]
4500 [0.0074993623, 0.0001711171, 0.0072336425, 0.0002654476, 2.7254612e-07]
5000 [0.007410524, 0.00030210894, 0.007146177, 0.0002640788, 2.6830224e-07]
5500 [0.0071797706, 0.0002568847, 0.0069562974, 0.00022320796, 2.65307e-07]
6000 [0.0066672754, 0.00010174669, 0.006467

# Testing

In [33]:
a = tf.constant([-1, 2, 3.0, 1.5, -4.5])

In [34]:
with tf.Session() as sess:
    print(sess.run(tf.maximum(a, tf.zeros_like(a))))
    print(sess.run(tf.nn.relu(a)))
    print(sess.run(a))

[0.  2.  3.  1.5 0. ]
[0.  2.  3.  1.5 0. ]
[-1.   2.   3.   1.5 -4.5]


In [27]:
import numpy as np

In [28]:
a = np.random.randn(5,1)
a

array([[ 1.08940902],
       [-0.99813817],
       [-0.00599837],
       [ 0.2449336 ],
       [-1.76236077]])

In [29]:
np.maximum(a, np.zeros_like(a))

array([[1.08940902],
       [0.        ],
       [0.        ],
       [0.2449336 ],
       [0.        ]])